# Imports e constantes

In [1]:
import sys
sys.path.append('../semana1/')
from aux_functions import read_data
from CNN_Net import Net
import ensemble_methods
from itertools import combinations
import random
from joblib import dump, load
from pathlib import Path
from torchvision import datasets, transforms

import tensorflow as tf
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold


from pytorch_metric_learning import distances, losses, reducers, testers
import numpy as np
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

import datetime
now = datetime.datetime.now()
timestamp = now.strftime('%Y-%m-%d')

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
SEED = 42
np.random.seed = SEED
random.seed = SEED
tf.random.set_seed(SEED)
# D1_DATA_DIR = Path('../data/patches-filter_by_area/d1/') # dataset 1
# D2_DATA_DIR = Path('../data/patches-filter_by_area/d2/') # dataset 2
# DATA_DIR = Path('./data/v1-and-fba/')
EXP_NAME = 'v1-ensble'
MODEL_OUT_DIR = Path(f'models/{EXP_NAME}_{timestamp}')
MODEL_OUT_DIR.mkdir(parents=True, exist_ok=False)
TRAIN_SIZE = .7
BATCH_SIZE = 128
LEARNING_RATE = 0.01
EPOCHS = 10
EMBEDDING_SIZE = 128
NUM_CLASSES = 2
IMG_SIZE = (96, 96)
NET_IMG_SIZE = (1, *IMG_SIZE)
LOSSES = ['Triplet', 'NPairs', 'CosFace', 'MultiSimilarity']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Dados mistos: Dataset v1+v2
v1 = 	melhoria de contraste na imagem inteira **(7341 imagens)** \
v2 =	melhoria de constraste em áreas específicas da imagem **(4277 imagens)** \
data aug = 	rotation_range=120, horizontal_flip=True, vertical_flip=True, zoom_range=0.3 **(10000 imagens)** \
**21390 exemplares da classe negativa**

**TOTAL: 21618 'LEISH' x 21390 'NO-LEISH'**

# Objetivos

- Definir a função de perda com base na escolha de loss_select.
- Inicializar o modelo de rede neural e o otimizador.
- Realizar o treinamento do modelo com os dados de treino por um número especificado de épocas.
- Gerar embeddings para os conjuntos de treino e teste.
- Utilizar esses embeddings para treinar e testar um classificador SVM printando no final, os melhores hiperparams do classificador (C e gamma) e o número de dimensões do PCA.
- Realizar a validação cruzada estratificada em vários splits para avaliar a estabilidade e confiabilidade do modelo.
- Calcular e imprimir a média e o desvio padrão do recall para as duas classes.
- Salvar o estado do modelo treinado para uso futuro, assim como o PCA e o Standard Scaler.

In [3]:
def save_model(loss_select, model, data_id):
    '''
    Salva o estado do modelo treinado em um arquivo.

    @params
    loss_select: Nome da função de perda utilizada durante o treinamento.
    model: Modelo de rede neural a ser salvo.
    data_id: identificador do dataset utilizado no treino.

    @returns
    None
    '''
    model_filename = MODEL_OUT_DIR.joinpath(f'model_{data_id}_{loss_select}_{EXP_NAME}_{timestamp}.pth')
    torch.save(model.state_dict(), model_filename)

In [4]:
def train(model, loss_func, device, train_loader, optimizer, epoch):
    '''
    Treina o modelo fornecido com o conjunto de dados de treinamento.

    @params
    model: O modelo de rede neural a ser treinado.
    loss_func: A função de perda a ser utilizada.
    device: O dispositivo no qual o modelo está sendo treinado (CPU ou GPU).
    train_loader: DataLoader contendo os dados de treinamento.
    optimizer: O otimizador usado para ajuste dos parâmetros do modelo.
    epoch: Número atual da época de treinamento.

    @returns
    Retorna a perda após a última iteração de treinamento na época atual.
    '''
    model.train()
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        loss = loss_func(embeddings, label)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(
                'Epoch {} Iteration {}: Loss = {}'.format(
                    epoch, batch_idx, loss
                )
            )

    return loss

def get_all_embeddings(dataset, model):
    '''
    Gera embeddings para todos os exemplos em um conjunto de dados usando o modelo fornecido.

    @params
    dataset: O conjunto de dados para o qual os embeddings serão gerados.
    model: Modelo de rede neural usado para gerar embeddings.

    @returns
    Retorna uma tupla contendo os embeddings e os rótulos correspondentes dos exemplos do conjunto de dados.
    '''
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

In [5]:
def train_model(loss_select, x_train_loader):
    '''
    Treina a CNN com base na função de perda especificada.

    @params
    loss_select: Uma string que especifica a função de perda a ser utilizada.
    x_train_loader: DataLoader contendo os dados de treinamento.

    @returns
    Retorna o modelo treinado, o otimizador e uma lista contendo as perdas de treinamento para cada época.
    '''
    loss_func = None
    if loss_select == 'Triplet':
        distance = distances.CosineSimilarity()
        reducer = reducers.ThresholdReducer(low=0)
        loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)

    if loss_select == 'NPairs':
        loss_func = losses.NPairsLoss()

    if loss_select == 'CosFace':
        loss_func = losses.CircleLoss()

    if loss_select == 'MultiSimilarity':
        loss_func = losses.MultiSimilarityLoss(alpha = 2, beta = 50, base=0.5)

    model = Net(EMBEDDING_SIZE, (1, *IMG_SIZE)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    lss_train = []
    for epoch in range(1, EPOCHS + 1):
        lss = train(model, loss_func, device, x_train_loader, optimizer, epoch)
        lss_train.append(lss)

    return model, optimizer, lss_train

def get_all_embeddings(dataset, model):
    '''
    Gera embeddings para todos os exemplos em um conjunto de dados usando o modelo fornecido.

    @params
    dataset: O conjunto de dados para o qual os embeddings serão gerados.
    model: Modelo de rede neural usado para gerar embeddings.

    @returns
    Retorna uma tupla contendo os embeddings e os rótulos correspondentes dos exemplos do conjunto de dados.
    '''
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

def train_eval_model(data_id, loss_select, train_set, test_set, model, C=None, gamma=None, perform_grid_search=False, to_save=False):
    '''
    Realiza Grid Search para hiperparâmetros do SVM se necessário, treina e avalia o modelo SVM.

    @params
    train_set: Conjunto de dados de treinamento.
    test_set: Conjunto de dados de teste.
    model: Modelo de rede neural para gerar embeddings.
    C: Parâmetro de regularização para o modelo SVM. Se None, é determinado via Grid Search.
    gamma: Parâmetro de kernel para o modelo SVM. Se None, é determinado via Grid Search.
    perform_grid_search: Booleano para realizar ou não o Grid Search.
    to_save: Booleano indicando se o modelo SVM e os pré-processadores devem ser salvos.

    @returns
    Retorna os rótulos verdadeiros de teste, as previsões feitas pelo modelo SVM, e os parâmetros C e gamma.
    '''
    print('Computing embeddings...')
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)

    scaler = StandardScaler()
    pca = PCA(n_components=0.9)
    train_embeddings = pca.fit_transform(scaler.fit_transform(train_embeddings.cpu().numpy()))
    test_embeddings = pca.transform(scaler.transform(test_embeddings.cpu().numpy()))

    if perform_grid_search:
        print('Performing GridSearch...')
        clf = svm.SVC(kernel='rbf')
        params = {'C': [1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001]}
        search = GridSearchCV(clf, params, verbose=3, scoring='recall_macro', n_jobs=-1)
        search.fit(train_embeddings, train_labels.cpu().numpy().ravel())
        C, gamma = search.best_params_['C'], search.best_params_['gamma']
        print(f'Best Params: C={C}, gamma={gamma}')
        print(f'PCA dim ={pca.n_components_}')

    clf = svm.SVC(C=C, gamma=gamma, kernel='rbf')
    clf.fit(train_embeddings, train_labels.cpu().numpy().ravel())
    predictions = clf.predict(test_embeddings)
    y_true_test = test_labels.cpu().numpy().ravel()
    print(f'RESULTS FOR LOSS = [{loss_select}]')
    print(classification_report(y_true_test, predictions))

    if to_save:
        dump(clf, MODEL_OUT_DIR.joinpath(f'clf_{data_id}_{loss_select}_{EXP_NAME}_{timestamp}.joblib'))
        dump(scaler, MODEL_OUT_DIR.joinpath(f'scaler_{data_id}_{loss_select}_{EXP_NAME}_{timestamp}.joblib'))
        dump(pca, MODEL_OUT_DIR.joinpath(f'pca_{data_id}_{loss_select}_{EXP_NAME}_{timestamp}.joblib'))
        print('~ Joblib files saved.')

    return y_true_test, predictions, C, gamma

def cross_val_model(data_id, loss_select, train_set, y_true_train, model, best_C, best_gamma, n_folds=15):
    '''
    Realiza a validação cruzada estratificada do modelo SVM com os melhores hiperparâmetros encontrados.

    @params
    loss_select: Nome da função de perda utilizada durante o treinamento.
    train_set: Conjunto de dados de treinamento. (torch.Dataset)
    model: Modelo de rede neural.
    best_C: Melhor valor de C encontrado para o SVM.
    best_gamma: Melhor valor de gamma encontrado para o SVM.
    n_folds: Número de folds para a validação cruzada.

    @returns
    None.
    '''
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True)
    iteration = 1
    recalls_fold_0 = []
    recalls_fold_1 = []

    for train_index, test_index in kf.split(train_set.indices, y_true_train):
        print(f'-------> RUN {iteration}')
        train_subset = torch.utils.data.Subset(train_set.dataset, train_index)
        test_subset = torch.utils.data.Subset(train_set.dataset, test_index)

        y_true_test, y_pred, *_ = train_eval_model(data_id, loss_select, train_subset, test_subset, model, C=best_C, gamma=best_gamma)

        report = classification_report(y_true_test, y_pred, output_dict=True)
        recalls_fold_0.append(report['0']['recall'])
        recalls_fold_1.append(report['1']['recall'])

        iteration += 1

    mean_recall_class_0 = np.mean(recalls_fold_0)
    mean_recall_class_1 = np.mean(recalls_fold_1)
    std_recall_class_0 = np.std(recalls_fold_0)
    std_recall_class_1 = np.std(recalls_fold_1)

    print(f'\n Loss ===== {loss_select}')
    print(f'Mean Recall (Class 0): {mean_recall_class_0}')
    print(f'Mean Recall (Class 1): {mean_recall_class_1}')
    print(f'Std Recall (Class 0): {std_recall_class_0}')
    print(f'Std Recall (Class 1): {std_recall_class_1}')

## Função principal

In [6]:
def run_full_pipeline(data_id, loss_select, x_train, x_test, x_train_loader, y_true_train):
    '''
    Executa todo o pipeline de treinamento, avaliação e validação cruzada do modelo.

    @params
    loss_select: Nome da função de perda a ser utilizada. (string)
    x_train: Conjunto de dados de treinamento. (torch.Dataset)
    x_test: Conjunto de dados de teste. (torch.Dataset)
    x_train_loader: DataLoader para o conjunto de treinamento. (torch.DataLoader)

    @returns
    None
    '''
    model, optim, losses_training = train_model(loss_select, x_train_loader)
    *_, best_C, best_gamma = train_eval_model(data_id, loss_select, x_train, x_test, model, perform_grid_search=True, to_save=True)
    cross_val_model(data_id, loss_select, x_train, y_true_train, model, best_C, best_gamma)
    save_model(loss_select, model, data_id)

In [19]:
x_train, x_test, x_train_loader, x_test_loader, y_true_train, y_true_test = read_data(DATA_DIR, BATCH_SIZE, TRAIN_SIZE)
# run_full_pipeline(loss[2], x_train, x_test, x_train_loader, y_true_train)

KeyboardInterrupt: 

In [ ]:
for err in ['Triplet', 'NPairs', 'MultiSimilarity']:
    print('-------------------------------------------------------------------------------')
    print(f'----- LOSS FUNCTION FOR THIS ITERATION [{err.upper()}] -------------------------------')
    # run_full_pipeline(err, x_train, x_test, x_train_loader, y_true_train)

-------------------------------------------------------------------------------
----- LOSS FUNCTION FOR THIS ITERATION [TRIPLET] -------------------------------
Epoch 1 Iteration 0: Loss = 0.22253894805908203
Epoch 1 Iteration 100: Loss = 0.20213256776332855
Epoch 1 Iteration 200: Loss = 0.18958674371242523
Epoch 2 Iteration 0: Loss = 0.2143809199333191
Epoch 2 Iteration 100: Loss = 0.2253275066614151
Epoch 2 Iteration 200: Loss = 0.1925477236509323
Epoch 3 Iteration 0: Loss = 0.19972577691078186
Epoch 3 Iteration 100: Loss = 0.19101019203662872
Epoch 3 Iteration 200: Loss = 0.18721525371074677
Epoch 4 Iteration 0: Loss = 0.18178299069404602
Epoch 4 Iteration 100: Loss = 0.1869731843471527
Epoch 4 Iteration 200: Loss = 0.20011961460113525
Epoch 5 Iteration 0: Loss = 0.20003992319107056
Epoch 5 Iteration 100: Loss = 0.2003728747367859
Epoch 5 Iteration 200: Loss = 0.20032835006713867
Epoch 6 Iteration 0: Loss = 0.20014555752277374
Epoch 6 Iteration 100: Loss = 0.2001161128282547
Epoch 6

100%|██████████| 404/404 [00:12<00:00, 31.85it/s]


Performing GridSearch...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Params: C=10, gamma=0.1
PCA dim =3
RESULTS FOR LOSS = [Triplet]
              precision    recall  f1-score   support

           0       0.71      0.85      0.78      6573
           1       0.81      0.65      0.72      6330

    accuracy                           0.75     12903
   macro avg       0.76      0.75      0.75     12903
weighted avg       0.76      0.75      0.75     12903

~ Joblib files saved.
-------> RUN 1
Computing embeddings...


100%|██████████| 63/63 [00:07<00:00,  7.94it/s]


RESULTS FOR LOSS = [Triplet]
              precision    recall  f1-score   support

           0       0.80      0.95      0.87      1456
           1       0.73      0.36      0.49       551

    accuracy                           0.79      2007
   macro avg       0.77      0.66      0.68      2007
weighted avg       0.78      0.79      0.76      2007

-------> RUN 2
Computing embeddings...


...

RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      1456
           1       0.80      0.84      0.82       551

    accuracy                           0.90      2007
   macro avg       0.87      0.88      0.87      2007
weighted avg       0.90      0.90      0.90      2007

-------> RUN 14
Computing embeddings...


100%|██████████| 63/63 [00:05<00:00, 11.25it/s]


RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      1487
           1       0.79      0.83      0.81       520

    accuracy                           0.90      2007
   macro avg       0.87      0.88      0.87      2007
weighted avg       0.90      0.90      0.90      2007

-------> RUN 15
Computing embeddings...


100%|██████████| 63/63 [00:06<00:00,  9.98it/s]


RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.93      0.90      0.92      1375
           1       0.80      0.85      0.83       632

    accuracy                           0.89      2007
   macro avg       0.87      0.88      0.87      2007
weighted avg       0.89      0.89      0.89      2007


 Loss ===== MultiSimilarity
Mean Recall (Class 0): 0.9138180366645953
Mean Recall (Class 1): 0.8512832554698557
Std Recall (Class 0): 0.00686276698384913
Std Recall (Class 1): 0.014493552737428248


## Retreinando para patches individuais do Dataset 1 e 2

## Dataset 1
fzr downsampling da classe negativa proporção (1:2) \
reduzir 7841 exemplos, contagem final 17018

In [10]:
import os, random
def delete_rand(n_files, data_dir):
    no_leish_files = os.listdir(data_dir)
    files_to_delete = random.sample(no_leish_files, n_files)

    count = 0
    for f in files_to_delete:
        os.remove(data_dir.joinpath(f))
        count += 1

    print(f'~ {count} arquivos removidos da pasta {data_dir}.')

In [8]:
D1_DATA_DIR = Path('../data/d1-patches/')
NO_LEISH_DATA_DIR = D1_DATA_DIR.joinpath('no-leish')
EXP_NAME = 'v1-ensble'
MODEL_OUT_DIR = Path(f'models/{EXP_NAME}_{timestamp}')
# MODEL_OUT_DIR.mkdir(parents=True, exist_ok=False)

# delete_rand(7841, NO_LEISH_DATA_DIR)

In [9]:
x_train, x_test, x_train_loader, x_test_loader, y_true_train, y_true_test = read_data(D1_DATA_DIR, BATCH_SIZE, TRAIN_SIZE)

label format = {'leish': 0, 'no-leish': 1}
train test split proportion = train[17868], test[7659]
leish in training set = 5972
leish in testing set = 2537


In [13]:
for err in LOSSES:
    print('-------------------------------------------------------------------------------')
    print(f'----- LOSS FUNCTION FOR THIS ITERATION [{err.upper()}] -------------------------------')
    run_full_pipeline('D1', err, x_train, x_test, x_train_loader, y_true_train)

-------------------------------------------------------------------------------
----- LOSS FUNCTION FOR THIS ITERATION [TRIPLET] -------------------------------
Epoch 1 Iteration 0: Loss = 0.2301642894744873
Epoch 1 Iteration 100: Loss = 0.20047864317893982
Epoch 2 Iteration 0: Loss = 0.19739511609077454
Epoch 2 Iteration 100: Loss = 0.2160094529390335
Epoch 3 Iteration 0: Loss = 0.19987818598747253
Epoch 3 Iteration 100: Loss = 0.20704802870750427
Epoch 4 Iteration 0: Loss = 0.21328607201576233
Epoch 4 Iteration 100: Loss = 0.20503541827201843
Epoch 5 Iteration 0: Loss = 0.20792824029922485
Epoch 5 Iteration 100: Loss = 0.20344966650009155
Epoch 6 Iteration 0: Loss = 0.2089468389749527
Epoch 6 Iteration 100: Loss = 0.2062797099351883
Epoch 7 Iteration 0: Loss = 0.20727601647377014
Epoch 7 Iteration 100: Loss = 0.19192494451999664
Epoch 8 Iteration 0: Loss = 0.20138540863990784
Epoch 8 Iteration 100: Loss = 0.1934385597705841
Epoch 9 Iteration 0: Loss = 0.20217996835708618
Epoch 9 Iter

100%|██████████| 240/240 [00:08<00:00, 27.89it/s]


Performing GridSearch...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Params: C=1, gamma=0.1
PCA dim =15
RESULTS FOR LOSS = [Triplet]
              precision    recall  f1-score   support

           0       0.83      0.86      0.84      2537
           1       0.93      0.91      0.92      5122

    accuracy                           0.89      7659
   macro avg       0.88      0.89      0.88      7659
weighted avg       0.90      0.89      0.89      7659

~ Joblib files saved.
-------> RUN 1
Computing embeddings...


100%|██████████| 38/38 [00:05<00:00,  7.05it/s]


RESULTS FOR LOSS = [Triplet]
              precision    recall  f1-score   support

           0       0.88      0.93      0.91       587
           1       0.93      0.88      0.90       605

    accuracy                           0.91      1192
   macro avg       0.91      0.91      0.91      1192
weighted avg       0.91      0.91      0.91      1192

-------> RUN 2
Computing embeddings...


...

RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       587
           1       0.92      0.90      0.91       604

    accuracy                           0.91      1191
   macro avg       0.91      0.91      0.91      1191
weighted avg       0.91      0.91      0.91      1191

-------> RUN 14
Computing embeddings...


100%|██████████| 38/38 [00:05<00:00,  7.04it/s]


RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.89      0.92      0.90       553
           1       0.93      0.90      0.91       638

    accuracy                           0.91      1191
   macro avg       0.91      0.91      0.91      1191
weighted avg       0.91      0.91      0.91      1191

-------> RUN 15
Computing embeddings...


100%|██████████| 38/38 [00:05<00:00,  7.16it/s]


RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.90      0.93      0.92       585
           1       0.93      0.91      0.92       606

    accuracy                           0.92      1191
   macro avg       0.92      0.92      0.92      1191
weighted avg       0.92      0.92      0.92      1191


 Loss ===== MultiSimilarity
Mean Recall (Class 0): 0.9206958614182199
Mean Recall (Class 1): 0.8945898483812579
Std Recall (Class 0): 0.009702782294887854
Std Recall (Class 1): 0.010250083324908555


## Ensemble

In [14]:
######################################## TRIPLET

triplet_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
triplet_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-22\model_D1_Triplet_v1-ensble_2023-11-22.pth'))

triplet_model.eval()

triplet_clf = load('./models/v1-ensble_2023-11-22\clf_D1_Triplet_v1-ensble_2023-11-22.joblib')
triplet_scaler = load('./models/v1-ensble_2023-11-22\scaler_D1_Triplet_v1-ensble_2023-11-22.joblib')
triplet_pca = load('./models/v1-ensble_2023-11-22\pca_D1_Triplet_v1-ensble_2023-11-22.joblib')
######################################## COSFACE

cosface_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
cosface_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-22\model_D1_CosFace_v1-ensble_2023-11-22.pth'))

cosface_model.eval()

cosface_clf = load('./models/v1-ensble_2023-11-22\clf_D1_CosFace_v1-ensble_2023-11-22.joblib')
cosface_scaler = load('./models/v1-ensble_2023-11-22\scaler_D1_CosFace_v1-ensble_2023-11-22.joblib')
cosface_pca = load('./models/v1-ensble_2023-11-22\pca_D1_CosFace_v1-ensble_2023-11-22.joblib')
######################################## NPAIRS

npairs_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
npairs_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-22\model_D1_NPairs_v1-ensble_2023-11-22.pth'))

npairs_model.eval()

npairs_clf = load('./models/v1-ensble_2023-11-22\clf_D1_NPairs_v1-ensble_2023-11-22.joblib')
npairs_scaler = load('./models/v1-ensble_2023-11-22\scaler_D1_NPairs_v1-ensble_2023-11-22.joblib')
npairs_pca = load('./models/v1-ensble_2023-11-22\pca_D1_NPairs_v1-ensble_2023-11-22.joblib')
######################################## MULTISIMILARITY

multisim_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
multisim_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-22\model_D1_MultiSimilarity_v1-ensble_2023-11-22.pth'))

multisim_model.eval()

multisim_clf = load('./models/v1-ensble_2023-11-22\clf_D1_MultiSimilarity_v1-ensble_2023-11-22.joblib')
multisim_scaler = load('./models/v1-ensble_2023-11-22\scaler_D1_MultiSimilarity_v1-ensble_2023-11-22.joblib')
multisim_pca = load('./models/v1-ensble_2023-11-22\pca_D1_MultiSimilarity_v1-ensble_2023-11-22.joblib')

In [15]:
classifiers = [triplet_clf, cosface_clf, multisim_clf, npairs_clf]
scalers = [triplet_scaler, cosface_scaler, multisim_scaler, npairs_scaler]
pcas = [triplet_pca, cosface_pca, multisim_pca, npairs_pca]
models = [triplet_model, cosface_model, multisim_model, npairs_model]

all_models_predictions = ensemble_methods.get_predictions(x_test, models, classifiers, scalers, pcas)

100%|██████████| 240/240 [00:07<00:00, 33.67it/s] 


In [16]:
model_str = ['triplet', 'cosface', 'multisim', 'npairs']
all_models_predictions = dict(zip(model_str, all_models_predictions))
all_models_predictions

{'triplet': array([0, 1, 1, ..., 1, 0, 0], dtype=int64),
 'cosface': array([0, 1, 1, ..., 1, 1, 0], dtype=int64),
 'multisim': array([0, 1, 1, ..., 1, 0, 0], dtype=int64),
 'npairs': array([1, 1, 1, ..., 0, 1, 1], dtype=int64)}

Majority voting \
precisa de número ímpar de modelos, ou seja, 4 combinações possíveis com 3 modelos

In [17]:
combs_k3 = list(combinations(all_models_predictions.items(), 3))

for comb in combs_k3:
    models_str, preds = zip(*comb)
    majority_pred = ensemble_methods.majority_voting(preds)

    print(f'RESULT MAJORITY VOTING [{models_str}]')
    print(classification_report(y_true_test, majority_pred, target_names=['Leish', 'No Leish']))


RESULT MAJORITY VOTING [('triplet', 'cosface', 'multisim')]
              precision    recall  f1-score   support

       Leish       0.88      0.88      0.88      2537
    No Leish       0.94      0.94      0.94      5122

    accuracy                           0.92      7659
   macro avg       0.91      0.91      0.91      7659
weighted avg       0.92      0.92      0.92      7659

RESULT MAJORITY VOTING [('triplet', 'cosface', 'npairs')]
              precision    recall  f1-score   support

       Leish       0.88      0.87      0.88      2537
    No Leish       0.94      0.94      0.94      5122

    accuracy                           0.92      7659
   macro avg       0.91      0.91      0.91      7659
weighted avg       0.92      0.92      0.92      7659

RESULT MAJORITY VOTING [('triplet', 'multisim', 'npairs')]
              precision    recall  f1-score   support

       Leish       0.86      0.86      0.86      2537
    No Leish       0.93      0.93      0.93      5122

    a

Weighted voting \
anteriormente, foi constatado que não havia melhora de perfomance entre 2 ou 3 modelos nessa modalidade, \
portanto, para priorizar o custo computacional, farei com apenas 2 modelos

In [18]:
combs_k2 = list(combinations(all_models_predictions.items(), 2))
weights = [0.4, 0.6]

for comb in combs_k2:
    models_str, preds = zip(*comb)
    weighted_pred_w1 = ensemble_methods.weighted_voting(preds, weights)
    print(f'RESULT WEIGHTED VOTING [{models_str}] / weights {weights}')
    print(classification_report(y_true_test, weighted_pred_w1, target_names=['Leish', 'No Leish']))

    weights.reverse()
    weighted_pred_w2 = ensemble_methods.weighted_voting(preds, weights)

    print(f'RESULT WEIGHTED VOTING [{models_str}] / weights {weights}')
    print(classification_report(y_true_test, weighted_pred_w2, target_names=['Leish', 'No Leish']))

RESULT WEIGHTED VOTING [('triplet', 'cosface')] / weights [0.4, 0.6]
              precision    recall  f1-score   support

       Leish       0.90      0.87      0.88      2537
    No Leish       0.94      0.95      0.94      5122

    accuracy                           0.92      7659
   macro avg       0.92      0.91      0.91      7659
weighted avg       0.92      0.92      0.92      7659

RESULT WEIGHTED VOTING [('triplet', 'cosface')] / weights [0.6, 0.4]
              precision    recall  f1-score   support

       Leish       0.83      0.86      0.84      2537
    No Leish       0.93      0.91      0.92      5122

    accuracy                           0.89      7659
   macro avg       0.88      0.89      0.88      7659
weighted avg       0.90      0.89      0.89      7659

RESULT WEIGHTED VOTING [('triplet', 'multisim')] / weights [0.6, 0.4]
              precision    recall  f1-score   support

       Leish       0.83      0.86      0.84      2537
    No Leish       0.93      

Stacking \
meta-classificador para aprender quando escolher o resultado de cada classificador, portanto, \
apenas uma combinação: todos os 4 modelos

In [20]:
meta_clf = ensemble_methods.stacking_ensemble(models, classifiers, scalers, pcas, x_train, y_true_train)
test_predictions = ensemble_methods.get_predictions(x_test, models, classifiers, scalers, pcas)
stacked_test_features = np.column_stack(test_predictions)
final_predictions = meta_clf.predict(stacked_test_features)

print(f'RESULT STACKING [All models] / meta-clf LogisticRegression')
print(classification_report(y_true_test, final_predictions, target_names=['Leish', 'No Leish']))

100%|██████████| 240/240 [00:07<00:00, 33.02it/s] 


RESULT STACKING [All models] / meta-clf LogisticRegression
              precision    recall  f1-score   support

       Leish       0.90      0.87      0.88      2537
    No Leish       0.94      0.95      0.94      5122

    accuracy                           0.92      7659
   macro avg       0.92      0.91      0.91      7659
weighted avg       0.92      0.92      0.92      7659



## Dataset 2
fzr downsampling da classe negativa proporção (1:2) \
reduzir 12264 exemplos, contagem final 26450

In [11]:
D2_DATA_DIR = Path('../data/d2-patches/')
NO_LEISH_DATA_DIR = D2_DATA_DIR.joinpath('no-leish')
# delete_rand(12264, NO_LEISH_DATA_DIR)

~ 12264 arquivos removidos da pasta ..\data\d2-patches\no-leish.


In [73]:
x_train, x_test, x_train_loader, x_test_loader, y_true_train, y_true_test = read_data(D2_DATA_DIR, BATCH_SIZE, TRAIN_SIZE)
for err in LOSSES:
    print('-------------------------------------------------------------------------------')
    print(f'----- LOSS FUNCTION FOR THIS ITERATION [{err.upper()}] -------------------------------')
    run_full_pipeline(err, x_train, x_test, x_train_loader, y_true_train)

label format = {'leish': 0, 'no-leish': 1}
train test split proportion = train[27772], test[11903]
leish in training set = 9238
leish in testing set = 3987
-------------------------------------------------------------------------------
----- LOSS FUNCTION FOR THIS ITERATION [TRIPLET] -------------------------------
Epoch 1 Iteration 0: Loss = 0.23032549023628235
Epoch 1 Iteration 100: Loss = 0.19984932243824005
Epoch 1 Iteration 200: Loss = 0.2006445825099945
Epoch 2 Iteration 0: Loss = 0.20070545375347137
Epoch 2 Iteration 100: Loss = 0.20014098286628723
Epoch 2 Iteration 200: Loss = 0.19986498355865479
Epoch 3 Iteration 0: Loss = 0.20056073367595673
Epoch 3 Iteration 100: Loss = 0.20002704858779907
Epoch 3 Iteration 200: Loss = 0.1972784698009491
Epoch 4 Iteration 0: Loss = 0.200966939330101
Epoch 4 Iteration 100: Loss = 0.19997265934944153
Epoch 4 Iteration 200: Loss = 0.20245866477489471
Epoch 5 Iteration 0: Loss = 0.20411135256290436
Epoch 5 Iteration 100: Loss = 0.200814738869667

100%|██████████| 372/372 [00:08<00:00, 44.67it/s] 


Performing GridSearch...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Params: C=10, gamma=0.1
PCA dim =4
RESULTS FOR LOSS = [Triplet]
              precision    recall  f1-score   support

           0       0.66      0.36      0.47      3987
           1       0.74      0.90      0.81      7916

    accuracy                           0.72     11903
   macro avg       0.70      0.63      0.64     11903
weighted avg       0.71      0.72      0.70     11903

~ Joblib files saved.
-------> RUN 1
Computing embeddings...


100%|██████████| 58/58 [00:05<00:00, 10.08it/s]


RESULTS FOR LOSS = [Triplet]
              precision    recall  f1-score   support

           0       0.68      0.67      0.67       921
           1       0.68      0.69      0.68       931

    accuracy                           0.68      1852
   macro avg       0.68      0.68      0.68      1852
weighted avg       0.68      0.68      0.68      1852

-------> RUN 2
Computing embeddings...


...

RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.93      0.87      0.90       906
           1       0.89      0.94      0.91       945

    accuracy                           0.91      1851
   macro avg       0.91      0.90      0.91      1851
weighted avg       0.91      0.91      0.91      1851

-------> RUN 14
Computing embeddings...


100%|██████████| 58/58 [00:05<00:00, 10.41it/s]


RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.92      0.88      0.90       864
           1       0.90      0.93      0.92       987

    accuracy                           0.91      1851
   macro avg       0.91      0.91      0.91      1851
weighted avg       0.91      0.91      0.91      1851

-------> RUN 15
Computing embeddings...


100%|██████████| 58/58 [00:05<00:00, 10.66it/s]


RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.93      0.88      0.90       892
           1       0.90      0.93      0.91       959

    accuracy                           0.91      1851
   macro avg       0.91      0.91      0.91      1851
weighted avg       0.91      0.91      0.91      1851


 Loss ===== MultiSimilarity
Mean Recall (Class 0): 0.8882425067880432
Mean Recall (Class 1): 0.9320866524426755
Std Recall (Class 0): 0.009697354362148896
Std Recall (Class 1): 0.008555886701134597


## Ensemble

In [76]:
######################################## TRIPLET
# recall 68/68, pca dim 4

triplet_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
triplet_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-22\model_Triplet_v1-ensble_2023-11-22.pth'))

triplet_model.eval()

triplet_clf = load('./models/v1-ensble_2023-11-22\clf_Triplet_v1-ensble_2023-11-22.joblib')
triplet_scaler = load('./models/v1-ensble_2023-11-22\scaler_Triplet_v1-ensble_2023-11-22.joblib')
triplet_pca = load('./models/v1-ensble_2023-11-22\pca_Triplet_v1-ensble_2023-11-22.joblib')
######################################## COSFACE
# recall 89/91, pca dim 40

cosface_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
cosface_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-22\model_CosFace_v1-ensble_2023-11-22.pth'))

cosface_model.eval()

cosface_clf = load('./models/v1-ensble_2023-11-22\clf_CosFace_v1-ensble_2023-11-22.joblib')
cosface_scaler = load('./models/v1-ensble_2023-11-22\scaler_CosFace_v1-ensble_2023-11-22.joblib')
cosface_pca = load('./models/v1-ensble_2023-11-22\pca_CosFace_v1-ensble_2023-11-22.joblib')
######################################## NPAIRS
# recall 60/64, pca dim 3

npairs_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
npairs_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-22\model_NPairs_v1-ensble_2023-11-22.pth'))

npairs_model.eval()

npairs_clf = load('./models/v1-ensble_2023-11-22\clf_NPairs_v1-ensble_2023-11-22.joblib')
npairs_scaler = load('./models/v1-ensble_2023-11-22\scaler_NPairs_v1-ensble_2023-11-22.joblib')
npairs_pca = load('./models/v1-ensble_2023-11-22\pca_NPairs_v1-ensble_2023-11-22.joblib')
######################################## MULTISIMILARITY
# recall 88/93, pca dim 22

multisim_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
multisim_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-22\model_MultiSimilarity_v1-ensble_2023-11-22.pth'))

multisim_model.eval()

multisim_clf = load('./models/v1-ensble_2023-11-22\clf_MultiSimilarity_v1-ensble_2023-11-22.joblib')
multisim_scaler = load('./models/v1-ensble_2023-11-22\scaler_MultiSimilarity_v1-ensble_2023-11-22.joblib')
multisim_pca = load('./models/v1-ensble_2023-11-22\pca_MultiSimilarity_v1-ensble_2023-11-22.joblib')

In [12]:
classifiers = [triplet_clf, cosface_clf, multisim_clf, npairs_clf]
scalers = [triplet_scaler, cosface_scaler, multisim_scaler, npairs_scaler]
pcas = [triplet_pca, cosface_pca, multisim_pca, npairs_pca]
models = [triplet_model, cosface_model, multisim_model, npairs_model]

all_models_predictions = ensemble_methods.get_predictions(x_test, models, classifiers, scalers, pcas)
all_models_predictions = dict(zip(model_str, all_models_predictions))
all_models_predictions

  0%|          | 0/612 [00:09<?, ?it/s]


KeyboardInterrupt: 

Majority Voting

In [78]:
combs_k3 = list(combinations(all_models_predictions.items(), 3))

for comb in combs_k3:
    models_str, preds = zip(*comb)
    majority_pred = ensemble_methods.majority_voting(preds)

    print(f'RESULT MAJORITY VOTING [{models_str}]')
    print(classification_report(y_true_test, majority_pred, target_names=['Leish', 'No Leish']))

RESULT MAJORITY VOTING [('triplet', 'cosface', 'multisim')]
              precision    recall  f1-score   support

       Leish       0.92      0.80      0.85      3987
    No Leish       0.91      0.96      0.93      7916

    accuracy                           0.91     11903
   macro avg       0.91      0.88      0.89     11903
weighted avg       0.91      0.91      0.91     11903

RESULT MAJORITY VOTING [('triplet', 'cosface', 'npairs')]
              precision    recall  f1-score   support

       Leish       0.86      0.49      0.62      3987
    No Leish       0.79      0.96      0.86      7916

    accuracy                           0.80     11903
   macro avg       0.82      0.72      0.74     11903
weighted avg       0.81      0.80      0.78     11903

RESULT MAJORITY VOTING [('triplet', 'multisim', 'npairs')]
              precision    recall  f1-score   support

       Leish       0.87      0.49      0.62      3987
    No Leish       0.79      0.96      0.87      7916

    a

Weighted Voting

In [79]:
combs_k2 = list(combinations(all_models_predictions.items(), 2))
weights = [0.4, 0.6]

for comb in combs_k2:
    models_str, preds = zip(*comb)
    weighted_pred_w1 = ensemble_methods.weighted_voting(preds, weights)
    print(f'RESULT WEIGHTED VOTING [{models_str}] / weights {weights}')
    print(classification_report(y_true_test, weighted_pred_w1, target_names=['Leish', 'No Leish']))

    weights.reverse()
    weighted_pred_w2 = ensemble_methods.weighted_voting(preds, weights)

    print(f'RESULT WEIGHTED VOTING [{models_str}] / weights {weights}')
    print(classification_report(y_true_test, weighted_pred_w2, target_names=['Leish', 'No Leish']))

RESULT WEIGHTED VOTING [('triplet', 'cosface')] / weights [0.4, 0.6]
              precision    recall  f1-score   support

       Leish       0.87      0.84      0.86      3987
    No Leish       0.92      0.94      0.93      7916

    accuracy                           0.91     11903
   macro avg       0.90      0.89      0.90     11903
weighted avg       0.91      0.91      0.91     11903

RESULT WEIGHTED VOTING [('triplet', 'cosface')] / weights [0.6, 0.4]
              precision    recall  f1-score   support

       Leish       0.66      0.36      0.47      3987
    No Leish       0.74      0.90      0.81      7916

    accuracy                           0.72     11903
   macro avg       0.70      0.63      0.64     11903
weighted avg       0.71      0.72      0.70     11903

RESULT WEIGHTED VOTING [('triplet', 'multisim')] / weights [0.6, 0.4]
              precision    recall  f1-score   support

       Leish       0.66      0.36      0.47      3987
    No Leish       0.74      

Stacking

In [80]:
meta_clf = ensemble_methods.stacking_ensemble(models, classifiers, scalers, pcas, x_train, y_true_train)
test_predictions = ensemble_methods.get_predictions(x_test, models, classifiers, scalers, pcas)
stacked_test_features = np.column_stack(test_predictions)
final_predictions = meta_clf.predict(stacked_test_features)

print(f'RESULT STACKING [All models] / meta-clf LogisticRegression')
print(classification_report(y_true_test, final_predictions, target_names=['Leish', 'No Leish']))

100%|██████████| 372/372 [00:08<00:00, 44.18it/s] 


RESULT STACKING [All models] / meta-clf LogisticRegression
              precision    recall  f1-score   support

       Leish       0.87      0.84      0.86      3987
    No Leish       0.92      0.94      0.93      7916

    accuracy                           0.91     11903
   macro avg       0.90      0.89      0.90     11903
weighted avg       0.91      0.91      0.91     11903



# Dataset 1+2

In [7]:
DPLUS_DATA_DIR = Path('../data/d1+d2-patches/')

In [8]:
x_train, x_test, x_train_loader, x_test_loader, y_true_train, y_true_test = read_data(DPLUS_DATA_DIR, BATCH_SIZE, TRAIN_SIZE)
for err in LOSSES:
    print('-------------------------------------------------------------------------------')
    print(f'----- LOSS FUNCTION FOR THIS ITERATION [{err.upper()}] -------------------------------')
    run_full_pipeline('D1', err, x_train, x_test, x_train_loader, y_true_train)

label format = {'leish': 0, 'no-leish': 1}
train test split proportion = train[45641], test[19561]
leish in training set = 15222
leish in testing set = 6512
-------------------------------------------------------------------------------
----- LOSS FUNCTION FOR THIS ITERATION [TRIPLET] -------------------------------
Epoch 1 Iteration 0: Loss = 0.2179347425699234
Epoch 1 Iteration 100: Loss = 0.19936731457710266
Epoch 1 Iteration 200: Loss = 0.20002858340740204
Epoch 1 Iteration 300: Loss = 0.19991640746593475
Epoch 2 Iteration 0: Loss = 0.20035287737846375
Epoch 2 Iteration 100: Loss = 0.19972048699855804
Epoch 2 Iteration 200: Loss = 0.20190778374671936
Epoch 2 Iteration 300: Loss = 0.19995476305484772
Epoch 3 Iteration 0: Loss = 0.20987917482852936
Epoch 3 Iteration 100: Loss = 0.23509901762008667
Epoch 3 Iteration 200: Loss = 0.20345167815685272
Epoch 3 Iteration 300: Loss = 0.24743512272834778
Epoch 4 Iteration 0: Loss = 0.20162254571914673
Epoch 4 Iteration 100: Loss = 0.211577653

100%|██████████| 612/612 [00:14<00:00, 41.08it/s]


Performing GridSearch...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Params: C=10, gamma=0.01
PCA dim =29
RESULTS FOR LOSS = [Triplet]
              precision    recall  f1-score   support

           0       0.84      0.83      0.84      6512
           1       0.92      0.92      0.92     13049

    accuracy                           0.89     19561
   macro avg       0.88      0.88      0.88     19561
weighted avg       0.89      0.89      0.89     19561

~ Joblib files saved.
-------> RUN 1
Computing embeddings...


100%|██████████| 96/96 [00:08<00:00, 10.79it/s]


RESULTS FOR LOSS = [Triplet]
              precision    recall  f1-score   support

           0       0.87      0.93      0.90      1455
           1       0.93      0.87      0.90      1588

    accuracy                           0.90      3043
   macro avg       0.90      0.90      0.90      3043
weighted avg       0.90      0.90      0.90      3043

-------> RUN 2
Computing embeddings...


...

RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.85      0.92      0.89      1474
           1       0.92      0.85      0.88      1568

    accuracy                           0.89      3042
   macro avg       0.89      0.89      0.89      3042
weighted avg       0.89      0.89      0.89      3042

-------> RUN 14
Computing embeddings...


100%|██████████| 96/96 [00:05<00:00, 16.56it/s]


RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.85      0.93      0.89      1490
           1       0.92      0.84      0.88      1552

    accuracy                           0.88      3042
   macro avg       0.89      0.89      0.88      3042
weighted avg       0.89      0.88      0.88      3042

-------> RUN 15
Computing embeddings...


100%|██████████| 96/96 [00:05<00:00, 16.53it/s]


RESULTS FOR LOSS = [MultiSimilarity]
              precision    recall  f1-score   support

           0       0.86      0.92      0.89      1452
           1       0.93      0.86      0.89      1590

    accuracy                           0.89      3042
   macro avg       0.89      0.89      0.89      3042
weighted avg       0.89      0.89      0.89      3042


 Loss ===== MultiSimilarity
Mean Recall (Class 0): 0.9249557700281391
Mean Recall (Class 1): 0.8535814874200035
Std Recall (Class 0): 0.0030220870546439045
Std Recall (Class 1): 0.006382162129424425


## Ensemble

In [9]:
######################################## TRIPLET

triplet_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
triplet_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-23\model_D1_Triplet_v1-ensble_2023-11-23.pth'))

triplet_model.eval()

triplet_clf = load('./models/v1-ensble_2023-11-23\clf_D1_Triplet_v1-ensble_2023-11-23.joblib')
triplet_scaler = load('./models/v1-ensble_2023-11-23\scaler_D1_Triplet_v1-ensble_2023-11-23.joblib')
triplet_pca = load('./models/v1-ensble_2023-11-23\pca_D1_Triplet_v1-ensble_2023-11-23.joblib')
######################################## COSFACE

cosface_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
cosface_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-23\model_D1_CosFace_v1-ensble_2023-11-23.pth'))

cosface_model.eval()

cosface_clf = load('./models/v1-ensble_2023-11-23\clf_D1_CosFace_v1-ensble_2023-11-23.joblib')
cosface_scaler = load('./models/v1-ensble_2023-11-23\scaler_D1_CosFace_v1-ensble_2023-11-23.joblib')
cosface_pca = load('./models/v1-ensble_2023-11-23\pca_D1_CosFace_v1-ensble_2023-11-23.joblib')
######################################## NPAIRS
# recall 60/64, pca dim 3

npairs_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
npairs_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-23\model_D1_NPairs_v1-ensble_2023-11-23.pth'))

npairs_model.eval()

npairs_clf = load('./models/v1-ensble_2023-11-23\clf_D1_NPairs_v1-ensble_2023-11-23.joblib')
npairs_scaler = load('./models/v1-ensble_2023-11-23\scaler_D1_NPairs_v1-ensble_2023-11-23.joblib')
npairs_pca = load('./models/v1-ensble_2023-11-23\pca_D1_NPairs_v1-ensble_2023-11-23.joblib')
######################################## MULTISIMILARITY
# recall 88/93, pca dim 22

multisim_model = Net(EMBEDDING_SIZE, NET_IMG_SIZE).to(device)
multisim_model.load_state_dict(torch.load('./models/v1-ensble_2023-11-23\model_D1_MultiSimilarity_v1-ensble_2023-11-23.pth'))

multisim_model.eval()

multisim_clf = load('./models/v1-ensble_2023-11-23\clf_D1_MultiSimilarity_v1-ensble_2023-11-23.joblib')
multisim_scaler = load('./models/v1-ensble_2023-11-23\scaler_D1_MultiSimilarity_v1-ensble_2023-11-23.joblib')
multisim_pca = load('./models/v1-ensble_2023-11-23\pca_D1_MultiSimilarity_v1-ensble_2023-11-23.joblib')

In [13]:
classifiers = [triplet_clf, cosface_clf, multisim_clf, npairs_clf]
scalers = [triplet_scaler, cosface_scaler, multisim_scaler, npairs_scaler]
pcas = [triplet_pca, cosface_pca, multisim_pca, npairs_pca]
models = [triplet_model, cosface_model, multisim_model, npairs_model]
model_str = ['triplet', 'cosface', 'multisim', 'npairs']

all_models_predictions = ensemble_methods.get_predictions(x_test, models, classifiers, scalers, pcas)
all_models_predictions = dict(zip(model_str, all_models_predictions))
all_models_predictions

100%|██████████| 612/612 [00:14<00:00, 42.10it/s]


{'triplet': array([1, 1, 0, ..., 1, 1, 1], dtype=int64),
 'cosface': array([1, 0, 0, ..., 1, 1, 0], dtype=int64),
 'multisim': array([1, 1, 0, ..., 1, 1, 1], dtype=int64),
 'npairs': array([1, 1, 1, ..., 1, 1, 1], dtype=int64)}

Majority voting

In [17]:
combs_k3 = list(combinations(all_models_predictions.items(), 3))

for comb in combs_k3:
    models_str, preds = zip(*comb)
    majority_pred = ensemble_methods.majority_voting(preds)

    print(f'RESULT MAJORITY VOTING [{models_str}]')
    print(classification_report(y_true_test, majority_pred, target_names=['Leish', 'No Leish']))

RESULT MAJORITY VOTING [('triplet', 'cosface', 'multisim')]
              precision    recall  f1-score   support

       Leish       0.86      0.84      0.85      6512
    No Leish       0.92      0.93      0.93     13049

    accuracy                           0.90     19561
   macro avg       0.89      0.89      0.89     19561
weighted avg       0.90      0.90      0.90     19561

RESULT MAJORITY VOTING [('triplet', 'cosface', 'npairs')]
              precision    recall  f1-score   support

       Leish       0.87      0.78      0.82      6512
    No Leish       0.90      0.94      0.92     13049

    accuracy                           0.89     19561
   macro avg       0.88      0.86      0.87     19561
weighted avg       0.89      0.89      0.89     19561

RESULT MAJORITY VOTING [('triplet', 'multisim', 'npairs')]
              precision    recall  f1-score   support

       Leish       0.85      0.81      0.83      6512
    No Leish       0.91      0.93      0.92     13049

    a

Weighted Voting

In [15]:
combs_k2 = list(combinations(all_models_predictions.items(), 2))
weights = [0.4, 0.6]

for comb in combs_k2:
    models_str, preds = zip(*comb)
    weighted_pred_w1 = ensemble_methods.weighted_voting(preds, weights)
    print(f'RESULT WEIGHTED VOTING [{models_str}] / weights {weights}')
    print(classification_report(y_true_test, weighted_pred_w1, target_names=['Leish', 'No Leish']))

    weights.reverse()
    weighted_pred_w2 = ensemble_methods.weighted_voting(preds, weights)

    print(f'RESULT WEIGHTED VOTING [{models_str}] / weights {weights}')
    print(classification_report(y_true_test, weighted_pred_w2, target_names=['Leish', 'No Leish']))

RESULT WEIGHTED VOTING [('triplet', 'cosface')] / weights [0.4, 0.6]
              precision    recall  f1-score   support

       Leish       0.83      0.81      0.82      6512
    No Leish       0.91      0.92      0.91     13049

    accuracy                           0.88     19561
   macro avg       0.87      0.86      0.87     19561
weighted avg       0.88      0.88      0.88     19561

RESULT WEIGHTED VOTING [('triplet', 'cosface')] / weights [0.6, 0.4]
              precision    recall  f1-score   support

       Leish       0.84      0.83      0.84      6512
    No Leish       0.92      0.92      0.92     13049

    accuracy                           0.89     19561
   macro avg       0.88      0.88      0.88     19561
weighted avg       0.89      0.89      0.89     19561

RESULT WEIGHTED VOTING [('triplet', 'multisim')] / weights [0.6, 0.4]
              precision    recall  f1-score   support

       Leish       0.84      0.83      0.84      6512
    No Leish       0.92      

Stacking

In [16]:
meta_clf = ensemble_methods.stacking_ensemble(models, classifiers, scalers, pcas, x_train, y_true_train)
test_predictions = ensemble_methods.get_predictions(x_test, models, classifiers, scalers, pcas)
stacked_test_features = np.column_stack(test_predictions)
final_predictions = meta_clf.predict(stacked_test_features)

print(f'RESULT STACKING [All models] / meta-clf LogisticRegression')
print(classification_report(y_true_test, final_predictions, target_names=['Leish', 'No Leish']))

100%|██████████| 612/612 [00:14<00:00, 42.10it/s]


RESULT STACKING [All models] / meta-clf LogisticRegression
              precision    recall  f1-score   support

       Leish       0.83      0.81      0.82      6512
    No Leish       0.91      0.92      0.91     13049

    accuracy                           0.88     19561
   macro avg       0.87      0.86      0.87     19561
weighted avg       0.88      0.88      0.88     19561

